### Install and Import Dependencies

In [15]:
!pip3 install torch torchvision torchaudio
!pip3 install transformers beautifulsoup4 pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 30.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 29.9 MB/s eta 0:00:00


In [30]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re 
import pandas as pd
import numpy as np

### Instantiate Model

In [17]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

### Encode and Calculate Sentiment

In [18]:
tokens = tokenizer.encode('I loved this', return_tensors = 'pt')

In [20]:
result = model(tokens)

In [22]:
torch.argmax(result.logits)

tensor(4)

### Collect Reviews

In [23]:
r = requests.get('https://www.yelp.com/biz/mejico-sydney-2')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class' :regex})
reviews = [result.text for result in results]

### Load Reviews into DataFeame and Score

In [31]:
df = pd.DataFrame(np.array(reviews), columns = ['review'])
df

,review
0,The food is fresh and tasty. The scallop cevi...
1,The food was decent not great.. We had the gu...
2,"Food was okay, guacamole was below average. Se..."
3,The food and service here was really good. It...
4,Visiting from Texas and decided to give this r...
5,Don't come here expecting legit Mexican food b...
6,Out of all the restaurants that I tried in Syd...
7,"Great atmosphere, attentive service, solid mar..."
8,We came here on a Thursday night @ 5pm and by ...
9,Have been here twice and have absolutely loved...


In [32]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors = 'pt')
    result = model(tokens)
    return int(torch.argmax(result.logits)) + 1

In [34]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [35]:
df

,review,sentiment
0,The food is fresh and tasty. The scallop cevi...,4
1,The food was decent not great.. We had the gu...,2
2,"Food was okay, guacamole was below average. Se...",2
3,The food and service here was really good. It...,5
4,Visiting from Texas and decided to give this r...,5
5,Don't come here expecting legit Mexican food b...,3
6,Out of all the restaurants that I tried in Syd...,5
7,"Great atmosphere, attentive service, solid mar...",3
8,We came here on a Thursday night @ 5pm and by ...,4
9,Have been here twice and have absolutely loved...,5
